In [1]:
from model import *
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import cv2
# from dataset_builder import *


def sample_images(generator, source, idx):
    w_noise = np.random.normal(0, 1, (1, 14, 14, 1024))
    predicted = generator.predict([source, w_noise])
    im = np.uint8(predicted[0, ...] * 127.5 + 127.5) #[-1,1->0,255]
    im_source = np.uint8(source[0, ...] * 255)  # [0,1->0,255]

    
    real_height = predicted[0, ...] 
    # smooth
    real_height = cv2.GaussianBlur(real_height,(5,5),1.0)* 12.5 + 12.5 # -> 0-25cm
    # 225 * 225 grid * 2.5 cm grid size
    # Example 2D array of elevation, and constant gridsize
    elev = real_height[:,:]
    cellsize = 2.5
    px, py = np.gradient(elev, cellsize)
    slope = np.sqrt(px ** 2 + py ** 2)
    slope = np.abs(np.arctan(slope)/np.pi*2)
    
    im_c = np.concatenate((np.squeeze(im, axis=-1), np.uint8(slope*255), im_source[..., 0], im_source[..., 1]), axis=1)
    plt.imsave('./outputs_test/sketch_conversion' + str(idx) + '.png', im_c, cmap='terrain')
    
    score = 0.3 * np.std(slope) + 0.5 * np.mean(slope)**2 + 0.2 * np.mean(np.abs(elev-np.mean(elev)))/25
    print(score)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    z = elev/100
    x, y = np.meshgrid([cellsize/100*i for i in range(z.shape[0])], [cellsize/100*i for i in range(z.shape[1])])
    ax.plot_surface(x, y, z,  facecolors=plt.cm.gist_earth(z/0.25))
    ax.set_xlim([-0.5,6])
    ax.set_ylim([-0.5,6])
    ax.set_zlim([0,0.8])
    ax.invert_xaxis()
    ax.view_init(elev=35, azim=90)
    plt.savefig('./outputs_test/heightmap' + str(idx) + '.png',dpi=200)
    plt.close()
    
def test_gan():
    terrain_generator = load_model('terrain_generator62000.h5')
    for i in range(30):
        print(i)
        source1 = np.random.rand(225,225,1)<25/225/225
        source2 = np.random.rand(225,225,1)<25/225/225
        
        source = np.concatenate([source1,source2],axis=-1)[None]
#         print(source.shape)
        sample_images(terrain_generator, source, i)

In [2]:
with tf.device('/cpu:0'):
    test_gan()

0
0.03383643736230226
1
0.033981617764841925
2
0.03712635342384493
3
0.027894673882693094
4
0.03425641486180416
5
0.03223111744127589
6
0.04416031020846255
7
0.03461205847771051
8
0.03545880694179472
9
0.028098678748892096
10
0.024315248139567647
11
0.03531593410987033
12
0.03470241565099623
13
0.029444868184946165
14
0.03223401110794451
15
0.04230041471931001
16
0.040178509681992734
17
0.028058355449632484
18
0.03692369400846617
19
0.03250983151334962
20
0.03027611390880208
21
0.031260456927809925
22
0.03287530642086444
23
0.02673127727527492
24
0.02681870651671684
25
0.041628725534375875
26
0.03678831747896241
27
0.030910315002333058
28
0.029140490084941323
29
0.0348253190519813
